In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langsmith import traceable
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough


In [4]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",temperature=0.1)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = TextLoader("../files/chapter_three.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()


In [5]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        ("human", "{question}"),
    ]
)

chain = (
    {
        "context": retriver,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)

In [6]:
chain.invoke("Aaronson 은 유죄인가요?")


AIMessage(content='죄송합니다. 그 질문에 대한 답을 모르겠습니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-2e6ad3ef-ce76-4a61-b733-e56dcc03e0ac-0', usage_metadata={'input_tokens': 1912, 'output_tokens': 17, 'total_tokens': 1929, 'input_token_details': {'cache_read': 0}})

In [7]:
chain.invoke("그가 테이블에 어떤 메시지를 썼나요?")

AIMessage(content='그는 테이블의 먼지 위에 손가락으로 2+2=5라고 썼습니다.\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-f9891888-ab84-4d0c-9fb4-c4eb566177a2-0', usage_metadata={'input_tokens': 2100, 'output_tokens': 25, 'total_tokens': 2125, 'input_token_details': {'cache_read': 0}})

In [8]:
chain.invoke("Julia 는 누구인가요?")


AIMessage(content='죄송합니다. 답을 모르겠습니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-7f9f5d1b-dd4d-4196-9bba-634592edfa3d-0', usage_metadata={'input_tokens': 1964, 'output_tokens': 12, 'total_tokens': 1976, 'input_token_details': {'cache_read': 0}})